# Retrieval Augmented Generation (RAG)

RAG nos permite darle documentos o bases de datos a un LLM para que este pueda obtener informacion desde alli y responder preguntas de acuerdo a los datos que puede extraer de esos documentos.

In [ ]:
!pip install openai --upgrade --quiet
!pip install langchain --upgrade --quiet

In [ ]:
import openai

# Cargar token desde google drive
from google.colab import drive
drive.mount('/content/drive')
with open('drive/MyDrive/Tokens/openai.txt','r') as f:
    token = f.read()
    openai.api_key = token

In [ ]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(openai_api_key=token, temperature=0.0, model="gpt-3.5-turbo")

Como habiamos visto anteriormente en el curso podemos crear vectores a partir de texto o imagenes y luego los podemos comparar para ver cuales son mas similares. LangChain nos permite hacer esto de forma muy sencilla:

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [ ]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [ ]:
retriever.get_relevant_documents("where did harrison work?")

In [ ]:
retriever.get_relevant_documents("what do bears like to eat")

Podemos incorporar todo esto a un chat de forma sencilla:

In [ ]:
from langchain.schema.runnable import RunnableMap

In [ ]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [ ]:
chain.invoke({"question": "where did harrison work?"})

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [ ]:
chain = prompt | model | output_parser

In [ ]:
chain.invoke({"topic": "bears"})

In [ ]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [ ]:
chain.invoke({"question": "where did harrison work?"})

Podemos utilizar base de datos mas potentes:

In [ ]:
#!pip install lark --upgrade --quiet

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'

In [ ]:
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [ ]:
print(vectordb._collection.count())

In [ ]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [ ]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [ ]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

Buscar similares:

In [ ]:
smalldb.similarity_search(question, k=2)

Buscar similares pero con diversidad:

In [ ]:
smalldb.max_marginal_relevance_search(question,k=2, fetch_k=3)

## Referencias:
Esta unidad es un resumen traducido con algunas modificaciones del [curso de Andrew Ng de Deeplearning.ai](https://www.deeplearning.ai/short-courses/langchain-chat-with-your-data/)

Meta, articulo original sobre RAG: https://ai.meta.com/blog/retrieval-augmented-generation-streamlining-the-creation-of-intelligent-natural-language-processing-models/

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/deeplearning/deeplearning.html)